In [123]:
import numpy as np
import pandas as pd 
# import holoviews as hv
# from holoviews import dim, opts
# import bokeh.io
# import iqplot
import urllib
import re
import helper
import random

In [124]:
# Run this to reload helper.py so you don't have to restart the kernel
import importlib

importlib.reload(helper)

<module 'helper' from '/Users/prakash/Desktop/Courses/ML/loan-sharks/poems/helper.py'>

## Preprocess Data

In [125]:
all_words, all_sequences, word_dict, all_sonnet_int, syllable_dict= helper.getAllWordsAndSequences("data/shakespeare.txt", "data/Syllable_dictionary.txt",syllable_count=True)

In [126]:
rhyme=[]
new_sonet_sequence=[]
for idx in range(len(all_sequences)):
    if len(all_sonnet_int[idx])>1:
        new_sonet_sequence.append(all_sonnet_int[idx][-2::-1])
    if idx%12==0:
        rhyme.append((all_sonnet_int[idx][-2],all_sonnet_int[idx+1][-2]))
    elif idx%4==0 and len(all_sonnet_int[idx+2])>1:
        rhyme.append((all_sonnet_int[idx][-2],all_sonnet_int[idx+2][-2]))
        rhyme.append((all_sonnet_int[idx+1][-2],all_sonnet_int[idx+3][-2]))

In [132]:
hmm5 = helper.unsupervised_HMM(new_sonet_sequence, 5, 300)

Iteration: 10
Iteration: 20
Iteration: 30
Iteration: 40
Iteration: 50
Iteration: 60
Iteration: 70
Iteration: 80
Iteration: 90
Iteration: 100
Iteration: 110
Iteration: 120
Iteration: 130
Iteration: 140
Iteration: 150
Iteration: 160
Iteration: 170
Iteration: 180
Iteration: 190
Iteration: 200
Iteration: 210
Iteration: 220
Iteration: 230
Iteration: 240
Iteration: 250
Iteration: 260
Iteration: 270
Iteration: 280
Iteration: 290
Iteration: 300


In [135]:
print('Sample Sonnet:\n====================')
sentence_rhyme=['' for i in range(14)]
for i in range(3):
    new_rhyme_1=random.choice(rhyme)
    new_rhyme_2=random.choice(rhyme)
    sentence_rhyme[i*4]=new_rhyme_1[0]
    sentence_rhyme[i*4+1]=new_rhyme_2[0]
    sentence_rhyme[i*4+2]=new_rhyme_1[1]
    sentence_rhyme[i*4+3]=new_rhyme_2[1]
new_rhyme_1=random.choice(rhyme)
sentence_rhyme[-2]=new_rhyme_1[0]
sentence_rhyme[-1]=new_rhyme_1[1]
for start_rhyme in sentence_rhyme:
    print(helper.sample_rev_sonents(hmm5, word_dict, syllable_dict, n_sonnet=10, start_word=start_rhyme))

Sample Sonnet:
Thee still spirits of you what no shore torn
Is three in hearts self yea cover i dyed
Which thorns me truth is losing that looks thee
And you to slain to forgetful what hair
My own in you but large and be all cross
This seest cannot she the robe leaving grow
Or best for death at i of case the deeds
I but second thoughts assure in prove fair
Not swerving of common dost tell in tears
I for more store to save in cry abide
Art kind would in thy name betray doth fears
Even of which of not still adjunct slide
Than thee have entitled that well all stars
That ten have in one and i import boast
